# Custom view report per user

In [ ]:
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd
from pprint import pprint

In [ ]:
gis = GIS('https://www.arcgis.com', 'username', 'password')
gis

## Search for content
The default `max_items` is 10. Be sure to change that. Sorting can be done with `sort_field` and `sort_order`. These rely on basic fields from the item. View is not one of them. 
We want views. This is a property of the item, and is not exposed to `search` or `advanced_search`

In [ ]:
allContent = gis.content.search('', max_items=500, sort_field = 'type', sort_order="desc")
print(len(allContent))

## Bring results into a DataFrame for sorting

### Sort all content by views

In [ ]:
rows = []
for item in allContent:
    rows.append({
        "title": item.title,
        "item": item,
        "type": item.type,
        "numViews": item.numViews,
        "user": item.owner
    })
df = pd.DataFrame(rows)
df.sort_values(by="numViews", ascending=False).head(10)

### Sort user content by views

In [ ]:
userNames = [user.username for user in gis.users.search('')]
print(f'There are {len(userNames)} users')

In [ ]:
for user in userNames:
    userDF = df[df['user']==user].sort_values(by="numViews", ascending=False).head(5)
    print(user, ": ", userDF['title'].to_list())

### Clean up results through a Pandas DataFrame
Results are displayed sorted by views, grouped by users, in descending order from total views, then by title is tied.

#### First, sort the values by user, then views, then title

In [ ]:
df_sorted = df.sort_values(['user', 'numViews', 'title'], ascending=[True, False, True])
df_sorted

#### Next, extract those top 5 items per user

In [ ]:
top5_per_user = df_sorted.groupby('user', group_keys=False).head(5)
top5_per_user

#### Next, count the total number of views from top 5 items per user

In [ ]:
sum_views = (
    top5_per_user.groupby('user', as_index=False)['numViews']
    .sum()
    .rename(columns={'numViews': 'sumTop5Views'})
    .sort_values('sumTop5Views', ascending=False)
)
sum_views

#### Lastly, order the DataFrame by the total view count

This code also re-defines the index for the DataFrame so it tracks the desired order. Be aware that the two users with no items are not included in the DataFrame.

In [ ]:
ordered_users = sum_views['user']
result = (
    top5_per_user
    .set_index('user')
    .loc[ordered_users]
    .reset_index()
)
result